In [ ]:
# Install required packages
# Run this cell first if you don't have these packages installed
!pip install numpy matplotlib scikit-learn datasets torch


# Week 7 — Convolutional Neural Networks (Image Classification) - PyTorch Version

**Setup Instructions:**
1. Run the cell above to install required packages (if not already installed)
2. **Important:** If you get import errors after installing packages, restart the Jupyter kernel:
   - Go to `Kernel` → `Restart Kernel` in the menu
   - Or use the restart button in the toolbar
3. Then re-run the import cell below

**Note:** This notebook uses PyTorch instead of NumPy for CNN implementation. PyTorch provides automatic differentiation and GPU support, making it easier to build and train neural networks.

**Objectives**

- Load and visualize the CIFAR-100 dataset from HuggingFace.
- Build a PCA + Logistic Regression baseline classifier; compute evaluation metrics (e.g., accuracy).
- Implement a simple CNN classifier using PyTorch that barely beats or fails to beat the baseline.
- Build a deeper CNN model that achieves better performance.
- Train a CNN model on data‑augmented images and visualize augmentations.
- Explore an advanced CNN feature (e.g., global average pooling) and observe its impact.


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from utils import (
    show_result,
    load_cifar100_dataset,
    pca_logistic_baseline,
    test_exercise_7_pca,
    test_exercise_7_simple_cnn,
    test_exercise_7_proper_cnn,
    test_exercise_7_data_aug_cnn,
    test_exercise_7_advanced_cnn,
    accuracy
)

# Set random seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


## 1. CIFAR-100 Image Dataset

In this exercise, we'll use the CIFAR-100 dataset, a well-known benchmark dataset for image classification. CIFAR-100 contains 60,000 32×32 color images in 100 classes (100 fine-grained classes), with 6,000 images per class.

We'll load the dataset from HuggingFace and convert it to grayscale to simplify training and focus on the CNN architecture rather than computational complexity.

**Task:** Use the `load_cifar10_dataset` function from `utils.py` to load a subset of CIFAR-100, then visualize a few random samples from each class. Report the number of training and test examples.

**Hint:** Call `load_cifar10_dataset(n_train=50000, n_test=200, seed=0, grayscale=False)` to get a manageable subset. The function returns 5 values: train images, train labels, test images, test labels, and class names.


In [ ]:
# Load CIFAR-100 dataset
# Using the full dataset for high performance
X_train, y_train, X_test, y_test, class_names = load_cifar100_dataset(
    n_train=50000, n_test=10000, seed=0, grayscale=False
)
print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")
print(f"Image shape: {X_train.shape[1:]}")
print(f"Classes: {class_names}")

# Visualize a few random samples from the training set
fig, axes = plt.subplots(1, 5, figsize=(12, 2.5))
for ax in axes:
    idx = random.randint(0, len(X_train) - 1)
    ax.imshow(X_train[idx]) # RGB, no cmap needed
    ax.set_title(f"{class_names[y_train[idx]]}\n(label {y_train[idx]})")
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2. PCA + Logistic Regression Baseline

A simple yet strong baseline for image classification is to flatten each image into a vector, project it onto a lower‑dimensional subspace using **Principal Component Analysis (PCA)**, and then train a multinomial logistic regression classifier.

1. Flatten the training and test images (shape `(N, H*W)`).
2. Fit a PCA model on the training data and project both the training and test data into a lower‑dimensional space (e.g., 20 components).
3. Train a `LogisticRegression` classifier on the reduced features.
4. Evaluate the classifier using **accuracy** (the fraction of correct predictions).

**Task:** Complete the function `student_pca_baseline(...)` below to implement this baseline. It should return the test accuracy as a float in `[0,1]`.

**Hints:**
- Use `train_images.reshape(train_images.shape[0], -1)` to flatten images from (N, H, W) to (N, H*W)
- Import `PCA` from `sklearn.decomposition` and `LogisticRegression` from `sklearn.linear_model`
- Make sure `n_components` doesn't exceed the number of features (use `min(n_components, n_features)`)
- Use `pca.fit_transform()` for training data and `pca.transform()` for test data
- The `accuracy` function from `utils` computes the classification accuracy


In [ ]:
def student_pca_baseline(train_images, train_labels, test_images, test_labels, n_components=20):
    '''
    Implements a PCA + Logistic Regression baseline classifier.

    Parameters:
        train_images: numpy array of shape (N_train, H, W) with float32 values in [0,1].
        train_labels: numpy array of shape (N_train,) of integer labels.
        test_images: numpy array of shape (N_test, H, W).
        test_labels: numpy array of shape (N_test,).
        n_components: number of principal components to retain.

    Returns:
        Test accuracy as a float in [0,1].
    '''
    # TODO: flatten images, fit PCA, train LogisticRegression, compute accuracy
    raise NotImplementedError


In [ ]:
# Evaluate the PCA baseline implementation
res = test_exercise_7_pca(student_pca_baseline)
show_result("Exercise 1 – PCA Baseline", res)

# If implemented, you can also test on the dataset generated above
try:
    acc = student_pca_baseline(X_train, y_train, X_test, y_test, 20)
    print(f"PCA baseline accuracy on the synthetic dataset: {acc:.3f}")
except NotImplementedError:
    print("Implement student_pca_baseline above.")


## 3. Simple Convolutional Neural Network (PyTorch)

Convolutional neural networks (CNNs) process images by learning **filters** that extract local patterns. We'll start with a very small CNN using PyTorch:

- One convolutional layer with a few filters (e.g., 4 filters, each $3\times3$).
- Apply a non‑linear activation such as ReLU.
- Flatten the result and feed it into a linear layer to produce class logits.

For training, use cross‑entropy loss and an optimizer (e.g., SGD) for a few epochs. Because this network is very shallow and the dataset is small, it may perform worse than the PCA baseline.

**Task:** Complete the function `student_simple_cnn(...)` below. It should construct the described network using PyTorch's `nn.Module`, train it for a few epochs on the training set, and return the test accuracy.

**Hints:**
- Use `nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3)` for the convolutional layer
- Use `nn.ReLU()` for activation
- Use `nn.Flatten()` or `.view()` to flatten the feature maps
- Use `nn.Linear()` for the final classification layer
- Convert NumPy arrays to PyTorch tensors: `torch.from_numpy(arr).float()`
- Move tensors to device: `.to(device)`
- Use `nn.CrossEntropyLoss()` for loss and `optim.SGD()` for optimizer
- Remember to call `model.train()` during training and `model.eval()` during evaluation


In [ ]:
def student_simple_cnn(train_images, train_labels, test_images, test_labels, num_epochs=5, learning_rate=0.01):
    '''
    Build and train a simple CNN with one convolutional layer followed by a linear classifier using PyTorch.
    Use small filter sizes (e.g., 3x3) and a small number of filters (e.g., 4).

    Parameters:
        train_images: numpy array (N_train, H, W).
        train_labels: numpy array (N_train,).
        test_images: numpy array (N_test, H, W).
        test_labels: numpy array (N_test,).
        num_epochs: number of training epochs.
        learning_rate: step size for gradient descent.

    Returns:
        Test accuracy as a float.
    '''
    # TODO: implement PyTorch CNN model, training loop, and evaluation
    raise NotImplementedError


In [ ]:
# Evaluate the simple CNN implementation
res = test_exercise_7_simple_cnn(student_simple_cnn)
show_result("Exercise 2 – Simple CNN", res)

# Optional: test on the dataset generated above
try:
    acc = student_simple_cnn(X_train, y_train, X_test, y_test)
    print(f"Simple CNN accuracy: {acc:.3f}")
except NotImplementedError:
    print("Implement student_simple_cnn above.")


## 4. Deeper CNN (Improved Model)

To achieve high performance, implement a **10-layer deep CNN** (VGG-style). We will use deeper blocks, Batch Normalization, and a modern optimizer schedule.

**Suggested Architecture (10 Convolutions):**
- **Block 1:** [Conv32 → BN → ReLU] × 2 → MaxPool
- **Block 2:** [Conv64 → BN → ReLU] × 2 → MaxPool
- **Block 3:** [Conv128 → BN → ReLU] × 3 → MaxPool
- **Block 4:** [Conv256 → BN → ReLU] × 3 → MaxPool
- **Classifier:** Flatten → Dropout → Linear(1024->512) → ReLU → Dropout → Linear(512->10)

**Task:** Complete the function `student_proper_cnn(...)` below. Use `optim.AdamW` and `OneCycleLR` scheduler.

**Hints:**
- Use `nn.Sequential` for blocks.
- Use `padding=1` to preserve dimensions.
- `OneCycleLR` helps converge much faster.


In [ ]:
def student_proper_cnn(train_images, train_labels, test_images, test_labels, num_epochs=30, learning_rate=0.001):
    '''
    Build and train a very deep 10-layer CNN (VGG-style).
    Architecture:
    - Block 1: [Conv32 -> BN -> ReLU] x 2 -> MaxPool
    - Block 2: [Conv64 -> BN -> ReLU] x 2 -> MaxPool
    - Block 3: [Conv128 -> BN -> ReLU] x 3 -> MaxPool
    - Block 4: [Conv256 -> BN -> ReLU] x 3 -> MaxPool
    - Classifier: Flatten -> Dropout -> Linear -> ReLU -> Dropout -> Linear

    Parameters:
        train_images: numpy array (N_train, H, W).
        train_labels: numpy array (N_train,).
        test_images: numpy array (N_test, H, W).
        test_labels: numpy array (N_test,).
        num_epochs: number of training epochs.
        learning_rate: step size for gradient descent.

    Returns:
        Test accuracy.
    '''
    # TODO: implement 10-layer VGG-style CNN
    raise NotImplementedError


In [ ]:
# Evaluate the deeper CNN implementation
res = test_exercise_7_proper_cnn(student_proper_cnn)
show_result("Exercise 3 – Proper CNN", res)

# Optional: test on the dataset generated above
try:
    acc = student_proper_cnn(X_train, y_train, X_test, y_test)
    print(f"Proper CNN accuracy: {acc:.3f}")
except NotImplementedError:
    print("Implement student_proper_cnn above.")


## 5. CNN with Data Augmentation

Data augmentation generates new training examples by applying random transformations to the original images. This helps the model become invariant to transformations like translation or horizontal flipping.

Typical augmentations for simple shape images include:

- Random horizontal flips.
- Random small shifts in position.
- Adding a bit of random noise.

**Task:** Complete the function `student_data_aug_cnn(...)` below. Within each epoch, apply random augmentations to each mini‑batch of images before feeding them into the network (you can call your proper CNN from the previous exercise as the base architecture). Return the test accuracy.

(Optional) Visualize a few examples of the original and augmented images.

**Hints:**
- Use `torch.flip(img, dims=[2])` for horizontal flip (assuming shape is N, C, H, W)
- Use `torch.roll(img, shifts=shift, dims=2)` to shift images vertically, `dims=3` for horizontal
- Add noise with `img + torch.randn_like(img) * noise_scale` and clip to [0, 1] using `torch.clamp()`
- Apply augmentations in each epoch BEFORE the forward pass
- Use the same two-layer CNN architecture from the previous exercise
- Only augment training data, NOT test data
- You can use NumPy for augmentation and convert to PyTorch tensors, or use PyTorch operations directly


In [ ]:
def student_data_aug_cnn(train_images, train_labels, test_images, test_labels, num_epochs=10, learning_rate=0.01):
    '''
    Train a CNN on augmented data using PyTorch.

    You can reuse your two‑layer CNN architecture from the previous section.
    Apply random augmentations (flips, shifts, noise) to the training images during training.
    Do not augment the test set.

    Parameters:
        train_images: numpy array (N_train, H, W).
        train_labels: numpy array (N_train,).
        test_images: numpy array (N_test, H, W).
        test_labels: numpy array (N_test,).
        num_epochs: number of training epochs.
        learning_rate: step size for gradient descent.

    Returns:
        Test accuracy.
    '''
    # TODO: implement data augmentation and training with PyTorch
    raise NotImplementedError


In [ ]:
# Evaluate the data‑augmented CNN implementation
res = test_exercise_7_data_aug_cnn(student_data_aug_cnn)
show_result("Exercise 4 – Data‑Augmented CNN", res)

# Optional: test on the dataset generated above
try:
    acc = student_data_aug_cnn(X_train, y_train, X_test, y_test)
    print(f"Data‑augmented CNN accuracy: {acc:.3f}")
except NotImplementedError:
    print("Implement student_data_aug_cnn above.")


## 6. Advanced CNN Feature: Global Average Pooling

One way to reduce the number of parameters in a CNN is to replace the flatten operation with **global average pooling**. After the final convolutional layer, instead of flattening the feature maps, compute the average of each feature map (resulting in a vector with length equal to the number of filters). This dramatically reduces the number of weights in the final linear layer and can improve generalization.

**Task:** Complete `student_advanced_cnn(...)` below. Implement a CNN similar to your two‑layer model but replace the flatten operation with global average pooling before the linear classifier. Train the network and return the test accuracy.

**Hints:**
- Use `nn.AdaptiveAvgPool2d(1)` for global average pooling (reduces (N, C, H, W) to (N, C, 1, 1))
- Or use `F.adaptive_avg_pool2d(x, 1)` or manually compute `torch.mean(x, dim=(2, 3), keepdim=True)`
- After pooling, use `.view(batch_size, -1)` or `.squeeze()` to get (N, C) shape
- The fully connected layer now takes n_filters inputs instead of n_filters * H * W
- This greatly reduces the number of parameters and can improve generalization
- Use the same two-layer conv architecture, just replace flatten with global average pooling


In [ ]:
def student_advanced_cnn(train_images, train_labels, test_images, test_labels, num_epochs=10, learning_rate=0.01):
    '''
    Build and train a CNN with global average pooling instead of flattening using PyTorch.

    After the final convolutional layer, compute the spatial average of each feature map. This
    reduces the dimensionality dramatically and can act as a regularizer.

    Parameters:
        train_images: numpy array (N_train, H, W).
        train_labels: numpy array (N_train,).
        test_images: numpy array (N_test, H, W).
        test_labels: numpy array (N_test,).
        num_epochs: number of training epochs.
        learning_rate: step size for gradient descent.

    Returns:
        Test accuracy.
    '''
    # TODO: implement CNN with global average pooling using PyTorch
    raise NotImplementedError


In [ ]:
# Evaluate the advanced CNN implementation
res = test_exercise_7_advanced_cnn(student_advanced_cnn)
show_result("Exercise 5 – Advanced CNN", res)

# Optional: test on the dataset generated above
try:
    acc = student_advanced_cnn(X_train, y_train, X_test, y_test)
    print(f"Advanced CNN accuracy: {acc:.3f}")
except NotImplementedError:
    print("Implement student_advanced_cnn above.")


## 7. Discussion

Briefly reflect on your results:

- Did the deeper CNN outperform the baseline models?
- How did data augmentation affect performance?
- What effect did global average pooling have?
- Why is it important to compare against simple baselines?
- How does PyTorch compare to implementing CNNs from scratch with NumPy?

_Provide your answers here._
